In [1]:
import sys, os
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity_v2/')
sys.path.insert(0, '/data1/home/rocks/discrete_elasticity_v2/python_src/')

import numpy as np
import scipy as sp
import scipy.optimize as opt
import numpy.random as rand
import gzip
import cPickle as pickle
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns
%matplotlib inline

sns.set(color_codes=True)
sns.set_context('poster', font_scale=1.75)
sns.set_palette("hls", 8)
# sns.set_palette("muted", 8)
sns.set_style('ticks', {'xtick.direction': 'in','ytick.direction': 'in', 'axes.linewidth': 2.0})


import mech_network_solver as mns
import network
import tuning_algs as talgs
import network_plot as nplot
import nonlin_solver as nls

Loading Mechanical Network Deformation Solver Module


In [3]:
DIM = 2

eta = 1
NTS = 20
Lp = -1.0
NN = 16
irec = 0

nw_label = "network_periodic_jammed/network_N{:05d}_Lp{:.2f}/network_irec{:04d}".format(NN, Lp, irec)

with gzip.open("/data1/home/rocks/data/{:}.pklz".format(nw_label), 'rb') as pkl_file:

    nw_data = pickle.load(pkl_file)
    net = nw_data['network']    

print net.NE
srate = np.zeros(net.NE, float)
    
for NTS in range(net.NE):
    print NTS
    for j in range(32):
        print j
        (success, inodesi, inodesj, onodesi, onodesj) = talgs.chooseFeasibleEdges(net, eta, NTS)
        if success:
            srate[NTS] += 1.0
            
srate /= 32.0

print srate



41
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
11
0
1
2
3
4
5
6
7
8
9
10
11

In [14]:
DIM = 2

eta = 1
NTS = 20
Lp = -1.0
NN = 8
irec = 0

nw_label = "network_periodic_jammed/network_N{:05d}_Lp{:.2f}/network_irec{:04d}".format(NN, Lp, irec)

with gzip.open("/data1/home/rocks/data/{:}.pklz".format(nw_label), 'rb') as pkl_file:

    nw_data = pickle.load(pkl_file)
    net = nw_data['network']    

NF = 1

edgei = net.edgei
edgej = net.edgej


inodesi = [[] for t in range(NF)]
inodesj = [[] for t in range(NF)]

onodesi = [[] for t in range(NF)]
onodesj = [[] for t in range(NF)]

edge = range(net.NE)

rand.shuffle(edge)

b = edge.pop()

inodesi[0].append(edgei[b])
inodesj[0].append(edgej[b])

for i in range(NTS):
    b = edge.pop()

    onodesi[0].append(edgei[b])
    onodesj[0].append(edgej[b])


print "inodes", inodesi, inodesj

print "onodes", onodesi, onodesj

isvec = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(inodesi[t], inodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        isvec[t].extend(bvec) 

istrain = [[] for t in range(NF)]
istrain[0].append(2.0)


osvec = [[] for t in range(NF)]
ostrain = [[] for t in range(NF)]
for t in range(NF):
    for (i, j) in zip(onodesi[t], onodesj[t]):
        posi = net.node_pos[DIM*i:DIM*i+DIM]
        posj = net.node_pos[DIM*j:DIM*j+DIM]
        bvec = posj - posi
        bvec -= np.round(bvec/net.L)*net.L
        osvec[t].extend(bvec) 

        r = rand.randint(2)
        ostrain[t].append((2*r-1) * eta)

print ostrain

pert = []
meas = []
for t in range(NF):
    pert.append(talgs.Perturb())
    pert[t].setInputStrain(len(inodesi[t]), inodesi[t], inodesj[t], istrain[t], isvec[t])

    meas.append(talgs.Measure())
    meas[t].setOutputStrain(len(onodesi[t]), onodesi[t], onodesj[t], osvec[t])

obj_func = mns.CyIneqRatioChangeObjFunc(len(np.concatenate(ostrain)), net.NE, 
                                        np.zeros(len(np.concatenate(ostrain)), float), np.concatenate(ostrain))    


K_init = np.ones(net.NE, float) / net.eq_length


cypert = []
for p in pert:
    cypert.append(p.getCyPert())

cymeas = []
for m in meas:
    cymeas.append(m.getCyMeas())

solver = mns.CyLinSolver(net.getCyNetwork(), len(cypert), cypert, cymeas)
solver.setIntStrengths(K_init)


# tuner = talgs.TuneContLin(net, pert, meas, obj_func, K_init)

# data = tuner.tune()

print net.DIM * net.NN
print net.NE

inodes [[4]] [[7]]
onodes [[0, 1, 1, 0, 4, 6, 2, 2, 4, 0, 2, 1, 0, 1, 3, 5, 2, 0, 1, 3]] [[7, 5, 7, 3, 6, 7, 4, 6, 5, 4, 3, 2, 2, 3, 6, 6, 5, 5, 6, 7]]
[[-1, 1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, -1, -1, -1, 1, -1]]
16
21


In [15]:
(feasible, u, con_err) = solver.solveFeasability(obj_func)
print feasible
print con_err


[array(True, dtype=bool)]
[array([  4.44089210e-16,  -3.33066907e-16,  -2.22044605e-16,
         0.00000000e+00,  -2.22044605e-16,  -5.55111512e-16,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -2.22044605e-16,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00])]


In [1]:
AH = solver.getAugHessian()[0]
AM = solver.getAugMeasMat()[0]
Af = solver.getAugForce()[0]

print AH.shape
print AM.shape
print Af.shape

print Af

NameError: name 'solver' is not defined

In [2]:
c = np.ones(len(AH), float)
bounds = [(-np.inf, np.inf) for i in range(len(AH))]

print len(c)
print len(ul)

def callback(xk, **args):
    print args['nit'], args['pivot']

res = opt.linprog(c, A_ub=-AM.transpose(), b_ub=np.zeros(1, float), callback=callback,
                  A_eq=AH, b_eq=Af, bounds=bounds, 
                  options={"disp": True, "bland": True, 'maxiter': 100000, 'tol': 1e-1})

print res

NameError: name 'np' is not defined

In [12]:
opt.show_options(solver='linprog', method='simplex', disp=True)

Solve the following linear programming problem via a two-phase
simplex algorithm.

minimize:     c^T * x

subject to:   A_ub * x <= b_ub
              A_eq * x == b_eq

Parameters
----------
c : array_like
    Coefficients of the linear objective function to be minimized.
A_ub : array_like
    2-D array which, when matrix-multiplied by x, gives the values of the
    upper-bound inequality constraints at x.
b_ub : array_like
    1-D array of values representing the upper-bound of each inequality
    constraint (row) in A_ub.
A_eq : array_like
    2-D array which, when matrix-multiplied by x, gives the values of the
    equality constraints at x.
b_eq : array_like
    1-D array of values representing the RHS of each equality constraint
    (row) in A_eq.
bounds : array_like
    The bounds for each independent variable in the solution, which can take
    one of three forms::
    None : The default bounds, all variables are non-negative.
    (lb, ub) : If a 2-element sequence is provided, 